In [77]:
%load_ext autoreload 
%autoreload 2

import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


import math
import numpy as np
import pandas as pd
from numpy import array
from numpy import array, arange, abs as np_abs
from numpy.fft import rfft, rfftfreq
from math import sin, pi
from scipy import signal
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import importlib
import model as md
import view as vw

importlib.reload(md)
importlib.reload(vw)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'view' from 'c:\\Users\\Артем\\Desktop\\Вуз\\Аспирантура\\Диссертация\\Алгоритм\\Relaxation_frequency_phase_algorithm\\view\\__init__.py'>

In [78]:
dir = project_root
U_s_path = os.path.join(dir, "data","real","SNR","Signal", "RTB2004_CHAN1.csv")
I_s_path= os.path.join(dir, "data","real","SNR","Signal", "RTB2004_CHAN2.csv")

U_n_path = os.path.join(dir, "data","real","SNR","Noize", "RTB2004_CHAN1.csv")
I_n_path= os.path.join(dir, "data","real","SNR","Noize", "RTB2004_CHAN2.csv")

t_s,U_s = md.make_real_data_list(U_s_path)
t_s,I_s = md.make_real_data_list(I_s_path)
t_n,U_n = md.make_real_data_list(U_n_path)
t_n,I_n = md.make_real_data_list(I_n_path)

In [79]:
F,V = md.get_spectrum3(t_s,U_s)
F_peak_calc = F[np.argmax(V)]
T=1/F_peak_calc

T_counts=md.convert_to_counts(t_s,T)
window=T_counts*8

fs_s = 1 / np.mean(np.diff(t_s))
print(fs_s)
 
vw.show_I_U_3(t_s,I_s,U_s)

1250000000.0


In [80]:
F,V = md.get_spectrum3(t_n,U_n)
 
vw.show_I_U_3(t_n,I_n,U_n,0,window)

In [81]:
#SNR по отдельно записанным шуму и сигналу с шумом
def get_SNR(s,n):
    s = s - np.mean(s)
    n = n - np.mean(n)

    Vrms_signal = np.sqrt(np.mean(s**2))
    Vrms_noise  = np.sqrt(np.mean(n**2))

    SNR_dB = 20 * np.log10(Vrms_signal / Vrms_noise)
    print(f"SNR = {SNR_dB:.2f} dB")
    return SNR_dB

get_SNR(I_s,I_n)
get_SNR(U_s,U_n)

SNR = 25.71 dB
SNR = 30.18 dB


np.float64(30.177274973372114)

In [82]:
t_f, U_f=md.filter_elliptic_bandpass(t_s,U_s,F_peak_calc,200e3,2)
t_f, I_f=md.filter_elliptic_bandpass(t_s,I_s,F_peak_calc,200e3,2)

In [83]:
def spectral_snr_fixedband(x, fs, f0, bw):
    # интегрируем PSD в фиксированной полосе +/- bw
    f, Pxx = signal.welch(x, fs, nperseg=4096)
    sig_mask = (f >= (f0-bw)) & (f <= (f0+bw))
    P_sig = np.trapz(Pxx[sig_mask], f[sig_mask]) if np.any(sig_mask) else 0.0
    P_noise = np.trapz(Pxx, f) - P_sig
    return 10*np.log10(P_sig / (P_noise+1e-30)), P_sig, P_noise

In [84]:
spectral_snr_fixedband(U_s,fs_s,F_peak_calc,100e3)

C:\Users\Артем\AppData\Local\Temp\ipykernel_31472\1308129716.py:6: DeprecationWarning:

`trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.

C:\Users\Артем\AppData\Local\Temp\ipykernel_31472\1308129716.py:7: RuntimeWarning:

divide by zero encountered in log10



(np.float64(-inf), 0.0, np.float64(8.977049293759878))